# 본문 스크래핑

In [ ]:
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
import random
import urllib.request
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException, TimeoutException
from selenium import webdriver
from selenium.webdriver.common.by import By
# import chromedriver_auto


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
# options.add_argument('headless') # 창 숨기기


In [2]:
from user_agent import generate_user_agent, generate_navigator

print(generate_user_agent(device_type='desktop'))
print(generate_user_agent(os='win', device_type='desktop'))

navigator = generate_navigator()

random_user = generate_user_agent(os='win', device_type='desktop')
print(random_user)

In [2]:
from selenium.webdriver.chrome.options import Options

# chrome_options = Options()
# chrome_options.add_argument(random_user)
driver = webdriver.Chrome(executable_path ='./chromedriver_win32/chromedriver.exe',options = options)

driver.maximize_window()
driver.get("https://www.naver.com/")
driver.implicitly_wait(10)


In [3]:
url_df = pd.read_csv("./나무문화/2020/나무문화_링크_2020_6월.csv", encoding='UTF-8')
print(url_df.shape, url_df.columns)
url_df


(11727, 3) Index(['url', 'title', 'date'], dtype='object')


,url,title,date
0,https://blog.naver.com/yeojuhangul/222017304299,여주 힐링 여행지 2곳 소개합니다(여주 여행코스),2020.06.30.
1,https://blog.naver.com/sy990520/222017303058,대중가요는 시대를 담았다 [국립한글박물관 기자단] 기획특별전...,2020.06.30.
2,https://blog.naver.com/guryesns/222017303062,구례 백두대간 생태교육장 체험 프로그램 누려~~,2020.06.30.
3,https://blog.naver.com/shirayuki_111/222017303214,신비아파트 더블X: 6개의 예언 귀신 정리,2020.06.30.
4,https://blog.naver.com/jsshopholic/222017301970,인사동 브런치 한식 레스토랑 @ 스페이스오,2020.06.30.
...,...,...,...
11722,https://blog.naver.com/dlagustnkh/221986773931,[후불제상조] 올바른상조 믿을 수 있는 후불제상조 함께 하세요,2020.06.01.
11723,https://blog.naver.com/aa13245/221986771937,2020년 6월 1일 아웃풋,2020.06.01.
11724,https://blog.naver.com/saidsong/221986773077,생명연습 - 문학동네 한국문학전집 001,2020.06.01.
11725,https://blog.naver.com/lcwlimkong/221986770764,"힐링을 주는 녹지공간, 숲세권의 가치 증가",2020.06.01.


In [4]:
blog_links = url_df['url'].to_list()
print(len(blog_links))


11727


In [6]:
# df = pd.DataFrame(columns = {'url','본문'})
# df.to_csv("./나무문화/2020/나무문화_본문_2020_6월.csv", index=False, encoding='UTF-8')
# df.to_excel("./나무문화/2020/나무문화_본문_2020_6월.xlsx", index=False, encoding='UTF-8')

In [5]:
df = pd.read_csv("./나무문화/2020/나무문화_본문_2020_6월.csv", encoding='UTF-8')
df

,본문,url
0,여주 힐링 여행지 2곳 (여주 여행코스)\n1. 황학산수목원\n세종대왕의 영릉과 명...,https://blog.naver.com/yeojuhangul/222017304299
1,안녕하세요 국립한글박물관 기자단 5기 조수연입니다.\n저번달에 이어 국립한글박물관 ...,https://blog.naver.com/sy990520/222017303058
2,"안녕하세요 여러분\n기운돋는 이름, 여기는 구례랍니다\n삶이 힘들고 지칠때\n구례지...",https://blog.naver.com/guryesns/222017303062
3,*오리지널 귀신 제외\n신비아파트 고스트볼X의 비밀 신화 속 귀신 정리\n<신비아파...,https://blog.naver.com/shirayuki_111/222017303214
4,50m\n© NAVER Corp.\n스페이스오\n서울특별시 종로구 인사동길 49 1...,https://blog.naver.com/jsshopholic/222017301970
...,...,...
10622,"[문학]\n존재하지 않는 이미지입니다.\n시절과 기분\n《여름, 스피드》로 많은 독...",https://blog.naver.com/readersguide/221991301449
10623,요즘 코로나 19 재확산으로\n궁이나 능 혹은 전시장 박물관이\n임시 휴관이더라구요...,https://blog.naver.com/ksjae0207/221991298644
10624,"안옥란이야기\n안옥란이야기 5,175개의 글\n안옥란이야기\n목록열기\n광교산 광교...",https://blog.naver.com/ooang40/221991293186
10625,세계를 바꾸는 착한마을 이야기\n저자\n박소명\n출판\n북멘토\n발매\n2014.0...,https://blog.naver.com/young_bong/221949817876


In [6]:
len(df)

10627

In [16]:
url_df.iloc[10627]

url      https://blog.naver.com/rpreview/221991286665
title                 Royce da 5'9" [The Allegory] 리뷰
date                                      2020.06.05.
Name: 10627, dtype: object

In [10]:
df.isnull().sum()

본문     2
url    0
dtype: int64

In [17]:
# 이건 필요할때 쓰면 됨. 블로그가 닫혔거나 삭제되었을때 빈 본문을 가진 데이터프레임 추가.
# content_dic = {'url' : ["https://blog.naver.com/rpreview/221991286665"], '본문' : [None]}
# additional_df = pd.DataFrame(content_dic)
# df = df.append(additional_df, ignore_index = True)

C:\Users\user\AppData\Local\Temp\ipykernel_1436\238471668.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)


In [ ]:
#### 삭제/비공개 링크 인덱스도 매번 바꿔줄 것!!! ###
# contents = []
# old_blogs = []
hide_link = []
waitsec = random.uniform(4,4.5)

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

for i in tqdm(blog_links[len(df):]):
    content_dic = {'url' : [], '본문' : []}
    content_dic['url'].append(i)
    time.sleep(waitsec)
    driver.get(i)
    driver.implicitly_wait(10)
    time.sleep(waitsec)
    try:
        driver.switch_to.frame("mainFrame")
    except TimeoutException as e:
        print('시간 초과 오류 : ', e)
        content_dic['본문'].append('수집불가능')
    except UnexpectedAlertPresentException as e:
    ### 이 오류 해결함. 에러 코드 파일에서 직접 임포트하면 된다!
        print(e)
                                                                            # 여기도 바꿔야 함
        print('{}번째 게시글은 비공개로 전환되었거나 게시판이 바뀜.'.format(blog_links.index(i)))
        hide_link.append(i)
        time.sleep(1.5)
        driver.switch_to.frame('mainFrame')
    try:
        a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
        content_dic['본문'].append(a)
    # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
    except NoSuchElementException:
        a = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
        content_dic['본문'].append(a)
        #old_blogs.append(i)
    additional_df = pd.DataFrame(content_dic)
    df = df.append(additional_df, ignore_index = True)
    df.to_csv("./나무문화/2020/나무문화_본문_2020_6월.csv", index=False, encoding='UTF-8')
    #df.to_excel("./나무문화/2020/나무문화_본문_2020_6월.xlsx", index=False, encoding='UTF-8')
    print(blog_links.index(i), df['본문'].tolist().index(a))
    
    display.clear_output(wait=True) # 출력지우기
    # if blog_links.index(i) != df['본문'].tolist().index(a):
    #     break


In [ ]:
df

In [ ]:
print(len(contents))
#print(len(old_blogs))
print(len(hide_link))
